In [1]:
from PyQt5.QtWidgets import *
from PyQt5.QtCore import Qt
import sys
import pandas as pd
import matplotlib.pyplot as plt
from io import BytesIO
from PyQt5.QtGui import QImage, QPixmap
from docx import Document
from docx.shared import Inches
import socket

In [2]:
class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        
        self.setWindowTitle("Citire fisier CSV")
        self.setGeometry(100, 100, 600, 400)
        
        # Crearea unui widget central pentru a adauga butonul
        central_widget = QWidget()               
        self.setCentralWidget(central_widget)    
        
        layout = QVBoxLayout(central_widget)
        
        #Layout orizontal pentru butoane
        button_layout = QHBoxLayout()
        
        self.button_read_csv = QPushButton("Citeste fisierul CSV", self)
        button_layout.addWidget(self.button_read_csv)

        self.button_generate_plot = QPushButton("Genereaza si afiseaza graficul", self)
        button_layout.addWidget(self.button_generate_plot)

        self.button_generate_docx = QPushButton("Genereaza fisier DOCX", self)
        button_layout.addWidget(self.button_generate_docx)

        layout.addLayout(button_layout)
        
        # Zona de afisare a graficului
        self.graph_display = QLabel(self)
        layout.addWidget(self.graph_display)

        # Variabila pentru stocarea datelor citite din fisierul CSV
        self.df = None
        
        self.button_read_csv.clicked.connect(self.read_csv_file)
        self.button_generate_plot.clicked.connect(self.generate_and_display_plot)
        self.button_generate_docx.clicked.connect(self.generate_docx_file)
        
        
    def read_csv_file(self):
        try:
            # Citirea fisierului CSV "populatie romania am.csv"
            self.df = pd.read_csv("populatie romania am.csv")
            print("Fisierul csv a fost citit cu succes.")
        except Exception as e:
            print("Eroare la citirea fisierului:", e)
            
    def generate_and_display_plot(self):
        if self.df is None:
            print("Fisierul csv nu a fost citit. Va fi citit acum...")
            self.read_csv_file()  # Daca df este None, citim fisierul CSV
        
        # Generarea si afisarea graficului
        plt.figure(figsize=(8, 5))
        plt.plot(self.df['An'], self.df['Populatie'], marker='o', linestyle='-', color='b')
        plt.xlabel('An')
        plt.ylabel('Populatie')
        plt.title('Evolutia populatiei in Romania 1960 - 2024')
        plt.grid(True)

        # Convertirea graficului intr-o imagine si afisarea in interfata
        buffer = BytesIO()
        plt.savefig(buffer, format='png')
        buffer.seek(0)

        image = QImage()
        image.loadFromData(buffer.getvalue())
        pixmap = QPixmap.fromImage(image)

        # Afisarea imaginii in zona de afisare a graficului
        self.graph_display.setPixmap(pixmap)

        # Inchiderea buffer-ului si a plot-ului
        buffer.close()
        plt.close()

    def generate_docx_file(self):
        if self.df is None:
            print("Fisierul csv nu a fost citit. Va fi citit acum...")
            self.read_csv_file()  # Daca df este None, citim fisierul CSV
            
        # Generarea graficului, daca nu a fost generat anterior
        if self.graph_display.pixmap() is None:
            print("Graficul nu a fost generat. Va fi generat acum...")
            self.generate_and_display_plot()
        
        try:
            # Crearea unui document docx
            doc = Document()
            
            # Adaugarea unui titlu
            doc.add_heading('Raport Populatie Romania', level=1)
            
            # Paragraf explicativ
            doc.add_paragraph('Acesta este un raport referitor la evolutia populatiei in Romania, '
                              'bazat pe datele din fisierul CSV "populatie romania am.csv".')
            
            # Adaugarea tabelului cu datele din fisierul CSV
            table = doc.add_table(rows=len(self.df) + 1, cols=len(self.df.columns))
            
            # Adaugarea antetului tabelului
            hdr_cells = table.rows[0].cells
            for i, col in enumerate(self.df.columns):
                hdr_cells[i].text = col
            
            # Adaugarea datelor in tabel
            for row in range(len(self.df)):
                for col in range(len(self.df.columns)):
                    table.cell(row + 1, col).text = str(self.df.iloc[row, col])
            
            # Salvarea imaginii graficului
            graph = "grafic.png"
            plt.figure(figsize=(8, 5))
            plt.plot(self.df['An'], self.df['Populatie'], marker='o', linestyle='-', color='b')
            plt.xlabel('An')
            plt.ylabel('Populatie')
            plt.title('Evolutia populatiei in Romania')
            plt.grid(True)
            plt.savefig(graph, format='png')
            plt.close()
            
            # Adaugarea imaginii in document
            doc.add_paragraph('\n')
            doc.add_picture(graph, width=Inches(6))
            
            # Salvarea documentului docx
            docx_filename = 'raport_populatie_romania.docx'
            doc.save(docx_filename)
            
            print("Fisierul DOCX a fost generat cu succes.")

            self.send_docx_file(docx_filename)
        except Exception as e:
            print("Eroare la generarea fisierului DOCX:", e)
    
    def send_docx_file(self, filename):
        HOST = "127.0.0.1"
        PORT = 34512

        BUFFER_SIZE = 65536
        
        with open(filename, 'rb') as file:
            docx_file = file.read(BUFFER_SIZE)
        with socket.socket( socket.AF_INET, socket.SOCK_STREAM ) as tcp_sock:
            tcp_sock.connect( (HOST, PORT) )
            
            # Trimiterea imaginii pe socket
            tcp_sock.send( docx_file )
        
            file.close()
            tcp_sock.close()

In [3]:
if __name__ == "__main__":
    # Crearea unei aplicatii PyQt5
    app = QApplication([])
  
    # Crearea unei instante a clasei MainWindow
    window = MainWindow()
    
    # Afisarea ferestrei
    window.show()
    
    # Executarea aplicatiei
    app.exec_()

Fisierul csv nu a fost citit. Va fi citit acum...
Fisierul csv a fost citit cu succes.
Graficul nu a fost generat. Va fi generat acum...
Fisierul DOCX a fost generat cu succes.
